In [1]:
# libraries
import os
import time

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
import seaborn as sns
import sklearn
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.model_selection import GroupKFold, KFold, RepeatedKFold
from xgboost import XGBRegressor

/home/balde/miniconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
emp = pd.read_csv("employment_status.csv")
ss = pd.read_csv("SampleSubmission.csv")
income = pd.read_csv("income_group.csv")
custom = pd.read_csv("customer.csv")
transaction = pd.read_csv("transactions.csv")

In [3]:
train.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE,DECLARED_NET_INCOME
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,2022-07,"13,500"
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,2022-04,"9,000"
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,2022-06,"8,000"
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,2022-08,"7,000"
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,2022-07,"10,400"


In [4]:
test.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,2022-06
1,ID_c28fb2d93c7b4be50a57aa7ebbe31a6bae33a18e1c6...,2022-01
2,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,2022-05
3,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,2022-05
4,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,2022-03


In [5]:
transaction.head()

,TRANSACTION_DESCRIPTION,CUSTOMER_IDENTIFIER,RECORD_DATE,ACCOUNT_TYPE_CODE,ACCOUNT_BALANCE,AMT,EVENT_NUMBER,PRODUCT_CODE,CHANNEL,ACCOUNT_NUMBER,TRANSACTION_REFERENCE
0,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-07-13 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,1857.56,1000.00,11785,SAVE,internet,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
1,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-02-26 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,8843.46,-2000.00,11348,SAVE,internet,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
2,POS PURCHASE,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-07-26 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,23.82,-199.98,148,CHEQ,system,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,04ae923a903fb07b5c846eb61eda25c4766800f02c2e09...
3,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-03-27 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,198.11,-79.39,11419,SAVE,system,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
4,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-05-06 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,13631.60,-200.00,11528,SAVE,atm,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN


In [6]:
transaction.shape, train.shape

((847544, 11), (3600, 3))

In [7]:
income.head()

,income_group_code,income_group_desc
0,0,NOT SUPPLIED
1,1,R 0 - 499
2,2,R500 - 999
3,3,"R1,000 - 1,999"
4,4,"R2,000 - 2,999"


In [8]:
income.shape

(22, 2)

In [9]:
test.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,2022-06
1,ID_c28fb2d93c7b4be50a57aa7ebbe31a6bae33a18e1c6...,2022-01
2,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,2022-05
3,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,2022-05
4,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,2022-03


## Here we are going to combine train and test set to better proprocess it.

here are few thing to keep in Mind:
    1. Data leakage is something that occurs when we concatinate train and test while separatin them again
    2. We should be careful about the shape of the train and the test on split

## Here I am going to use this method

Add an indicator column while concatenating the two dataframes, so you can later seperate them again:

df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])
Then later you can split them again:

test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]


from this (Link)['https://datascience.stackexchange.com/questions/81617/how-to-combine-and-separate-test-and-train-data-for-data-cleaning']

we are going to concatinate our train and test data

In [10]:
# data = pd.concat([train.assign(ind="train"), test.assign(ind="test")]).reset_index(
#     drop=True
# )

In [11]:
train["type"] = "train"
test["type"] = "test"

data = pd.concat([train, test])

In [12]:
test.shape, train.shape

((1544, 3), (3600, 4))

In [13]:
data.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE,DECLARED_NET_INCOME,type
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,2022-07,"13,500",train
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,2022-04,"9,000",train
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,2022-06,"8,000",train
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,2022-08,"7,000",train
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,2022-07,"10,400",train


In [14]:
data.shape

(5144, 4)

In [15]:
emp.head()

,employee_status_code,employee_status_desc
0,0,Resigned
1,3,Active
2,1,Inactive
3,2,Retiree


In [16]:
custom.head()

,CUSTOMER_IDENTIFIER,DATE_LAST_UPDATED,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE
0,ID_00a7d65f82a6819e79c29a84e805777f0265308b655...,2022-09,1,6,4,7,51
1,ID_09651edcac6c55a5e401062b0c0ef7bbba3f3e49538...,2022-09,0,7,10,3,32
2,ID_17215674e76eb33b47c4dd3ac3392e3e8f1a34b131b...,2022-09,1,7,7,1,41
3,ID_212860dd16c79ea8eae97cc1fc181e6afd4862fceaf...,2022-09,0,13,10,1,44
4,ID_24953f74da44bd0915c4cb78c1d97cbbd66e85bbd91...,2022-08,0,13,6,1,44


In [17]:
custom.shape

(5144, 7)

In [18]:
ss.head()

,CUSTOMER_IDENTIFIER,DECLARED_NET_INCOME
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,0
1,ID_c28fb2d93c7b4be50a57aa7ebbe31a6bae33a18e1c6...,0
2,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,0
3,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,0
4,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,0


As they told as that account number is duplicated so we are going to drop it to see if we can get the product code, channel, and amt columns on our train test.

In [19]:
transaction = transaction.drop_duplicates(["ACCOUNT_NUMBER"])
transaction.shape

(5587, 11)

In [20]:
transaction.head()

,TRANSACTION_DESCRIPTION,CUSTOMER_IDENTIFIER,RECORD_DATE,ACCOUNT_TYPE_CODE,ACCOUNT_BALANCE,AMT,EVENT_NUMBER,PRODUCT_CODE,CHANNEL,ACCOUNT_NUMBER,TRANSACTION_REFERENCE
0,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-07-13 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,1857.56,1000.00,11785,SAVE,internet,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
322,POS PURCHASE,ID_0925f20edcf26977988f43469681d58441b449a88c7...,2022-06-26 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,24.59,-38.91,50,CHEQ,system,0e77f4d244bd3833b4aa54d3ed788a1d99e127b1cf63fd...,c9681cfc2cec371c82a1d38bad5fc9416868c36d17d0df...
329,NaN,ID_09b52b96ac7bc78862e8340d3a7be0fc103227f9bcc...,2022-04-30 00:00:00+02:00,e17913461e7c03cd0f2c1ac41584dcd801cb32e7f86ad9...,2175.76,-33.99,1656,SAVE,system,7d7506943996ff370ac93bf43200a318fbf291013aea9a...,NaN
553,POS PURCHASE,ID_13da6738cf3191e70f74c127b14345e4ddc9bc04adc...,2021-12-09 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,2232.80,-51.50,14761,CHEQ,system,db1d38f2520c0faeb26893765d4184bdd1c94e987ccfea...,2156f36cc7b365fc16661bef94f3a303a3fd4c47bfa242...
904,NaN,ID_1af3755a6e5931439773fcb2ec4dda1190dfd2fece3...,2022-04-27 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,3102.59,-333.24,8005,SAVE,system,630aad4f576667271bf2848309e206465a343c8295480b...,NaN


Now i am going to select only the columns that I am interested in 

In [21]:
transaction = transaction.iloc[:5144, :]
transaction = transaction[
    [
        "CUSTOMER_IDENTIFIER",
        "ACCOUNT_BALANCE",
        "CHANNEL",
        "AMT",
        "PRODUCT_CODE",
        "RECORD_DATE",
    ]
]

transaction.reset_index(inplace=True, drop=True)
transaction.head()

,CUSTOMER_IDENTIFIER,ACCOUNT_BALANCE,CHANNEL,AMT,PRODUCT_CODE,RECORD_DATE
0,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,1857.56,internet,1000.00,SAVE,2022-07-13 00:00:00+02:00
1,ID_0925f20edcf26977988f43469681d58441b449a88c7...,24.59,system,-38.91,CHEQ,2022-06-26 00:00:00+02:00
2,ID_09b52b96ac7bc78862e8340d3a7be0fc103227f9bcc...,2175.76,system,-33.99,SAVE,2022-04-30 00:00:00+02:00
3,ID_13da6738cf3191e70f74c127b14345e4ddc9bc04adc...,2232.80,system,-51.50,CHEQ,2021-12-09 00:00:00+02:00
4,ID_1af3755a6e5931439773fcb2ec4dda1190dfd2fece3...,3102.59,system,-333.24,SAVE,2022-04-27 00:00:00+02:00


In [22]:
transaction.shape

(5144, 6)

# Merging with the Customer data

In [23]:
data = data.merge(transaction, on="CUSTOMER_IDENTIFIER")
data.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE_x,DECLARED_NET_INCOME,type,ACCOUNT_BALANCE,CHANNEL,AMT,PRODUCT_CODE,RECORD_DATE_y
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,2022-07,"13,500",train,35789.84,atm,-5000.0,CHEQ,2022-07-21 00:00:00+02:00
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,2022-04,"9,000",train,5896.51,system,-375.0,SAVE,2021-10-12 00:00:00+02:00
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,2022-06,"8,000",train,6328.70,atm,-1050.0,SAVE,2022-02-09 00:00:00+02:00
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,2022-08,"7,000",train,4571.16,system,-23.2,SAVE,2022-08-31 00:00:00+02:00
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,2022-07,"10,400",train,3334.72,atm,-100.0,CHEQ,2022-07-30 00:00:00+02:00


# Concatinating with the transaction data

In [24]:
data = data.merge(custom, on="CUSTOMER_IDENTIFIER")
data.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE_x,DECLARED_NET_INCOME,type,ACCOUNT_BALANCE,CHANNEL,AMT,PRODUCT_CODE,RECORD_DATE_y,DATE_LAST_UPDATED,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,2022-07,"13,500",train,35789.84,atm,-5000.0,CHEQ,2022-07-21 00:00:00+02:00,2022-07,0,15,7,1,45
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,2022-04,"9,000",train,5896.51,system,-375.0,SAVE,2021-10-12 00:00:00+02:00,2022-04,0,15,5,1,32
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,2022-06,"8,000",train,6328.70,atm,-1050.0,SAVE,2022-02-09 00:00:00+02:00,2022-08,0,7,7,7,59
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,2022-08,"7,000",train,4571.16,system,-23.2,SAVE,2022-08-31 00:00:00+02:00,2022-08,1,9,14,7,57
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,2022-07,"10,400",train,3334.72,atm,-100.0,CHEQ,2022-07-30 00:00:00+02:00,2022-09,0,12,11,1,29


In [25]:
data.shape

(5144, 15)

## We are going to fix the data type of the target variable it's categorical we are converting it into int type
Here there is a problem we notice that there is **,** in our target variable I got help from this one: [Convert to Numerical](https://stackoverflow.com/questions/55557004/getting-attributeerror-float-object-has-no-attribute-replace-error-while)

In [26]:
data["DECLARED_NET_INCOME"] = data["DECLARED_NET_INCOME"].astype(str)
data["DECLARED_NET_INCOME"] = [x.replace(",", "") for x in data["DECLARED_NET_INCOME"]]

In [27]:
data["DECLARED_NET_INCOME"] = data["DECLARED_NET_INCOME"].astype(float)
# data["DECLARED_NET_INCOME"].apply(lambda x: str(x.replace(",", "").strip()))

In [28]:
data.shape

(5144, 15)

In [29]:
data.dtypes

CUSTOMER_IDENTIFIER          object
RECORD_DATE_x                object
DECLARED_NET_INCOME         float64
type                         object
ACCOUNT_BALANCE             float64
CHANNEL                      object
AMT                         float64
PRODUCT_CODE                 object
RECORD_DATE_y                object
DATE_LAST_UPDATED            object
SEX_CODE                      int64
INCOME_GROUP_CODE             int64
NUMBER_OF_ACCOUNTS            int64
OCCUPATIONAL_STATUS_CODE      int64
AGE                           int64
dtype: object

In [30]:
data["RECORD_DATE"] = pd.to_datetime(data["RECORD_DATE_x"])
# data["YEAR"] = data["RECORD_DATE"].dt.year
data["MONTH"] = data["RECORD_DATE"].dt.month
# data["DAY"] = data["RECORD_DATE"].dt.day

In [31]:
# converting the target to numeric

# data["DECLARED_NET_INCOME"] = data["DECLARED_NET_INCOME"].astype(float)

In [32]:
data.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE_x,DECLARED_NET_INCOME,type,ACCOUNT_BALANCE,CHANNEL,AMT,PRODUCT_CODE,RECORD_DATE_y,DATE_LAST_UPDATED,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE,RECORD_DATE,MONTH
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,2022-07,13500.0,train,35789.84,atm,-5000.0,CHEQ,2022-07-21 00:00:00+02:00,2022-07,0,15,7,1,45,2022-07-01,7
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,2022-04,9000.0,train,5896.51,system,-375.0,SAVE,2021-10-12 00:00:00+02:00,2022-04,0,15,5,1,32,2022-04-01,4
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,2022-06,8000.0,train,6328.70,atm,-1050.0,SAVE,2022-02-09 00:00:00+02:00,2022-08,0,7,7,7,59,2022-06-01,6
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,2022-08,7000.0,train,4571.16,system,-23.2,SAVE,2022-08-31 00:00:00+02:00,2022-08,1,9,14,7,57,2022-08-01,8
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,2022-07,10400.0,train,3334.72,atm,-100.0,CHEQ,2022-07-30 00:00:00+02:00,2022-09,0,12,11,1,29,2022-07-01,7


In [33]:
data.drop(["RECORD_DATE_x", "DATE_LAST_UPDATED"], axis=1, inplace=True)
data.head()

,CUSTOMER_IDENTIFIER,DECLARED_NET_INCOME,type,ACCOUNT_BALANCE,CHANNEL,AMT,PRODUCT_CODE,RECORD_DATE_y,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE,RECORD_DATE,MONTH
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,13500.0,train,35789.84,atm,-5000.0,CHEQ,2022-07-21 00:00:00+02:00,0,15,7,1,45,2022-07-01,7
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,9000.0,train,5896.51,system,-375.0,SAVE,2021-10-12 00:00:00+02:00,0,15,5,1,32,2022-04-01,4
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,8000.0,train,6328.70,atm,-1050.0,SAVE,2022-02-09 00:00:00+02:00,0,7,7,7,59,2022-06-01,6
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,7000.0,train,4571.16,system,-23.2,SAVE,2022-08-31 00:00:00+02:00,1,9,14,7,57,2022-08-01,8
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,10400.0,train,3334.72,atm,-100.0,CHEQ,2022-07-30 00:00:00+02:00,0,12,11,1,29,2022-07-01,7


In [34]:
# train = data.iloc[:3600, :]
# train.shape

In [35]:
# test = data.iloc[3600:, :]
# test.reset_index(drop=True, inplace=True)
# test.head()

In [36]:
# test, train = data[data["ind"].eq("test")], data[data["ind"].eq("train")]

In [42]:
train = data[data["type"] == "train"]

test = data[data["type"] == "test"]

train.drop("type", axis=1, inplace=True)
test.drop("type", axis=1, inplace=True)

<ipython-input-42-90c469d1f67f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop("type", axis=1, inplace=True)
<ipython-input-42-90c469d1f67f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop("type", axis=1, inplace=True)


In [43]:
data.head()

,CUSTOMER_IDENTIFIER,DECLARED_NET_INCOME,type,ACCOUNT_BALANCE,CHANNEL,AMT,PRODUCT_CODE,RECORD_DATE_y,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE,RECORD_DATE,MONTH
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,13500.0,train,35789.84,atm,-5000.0,CHEQ,2022-07-21 00:00:00+02:00,0,15,7,1,45,2022-07-01,7
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,9000.0,train,5896.51,system,-375.0,SAVE,2021-10-12 00:00:00+02:00,0,15,5,1,32,2022-04-01,4
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,8000.0,train,6328.70,atm,-1050.0,SAVE,2022-02-09 00:00:00+02:00,0,7,7,7,59,2022-06-01,6
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,7000.0,train,4571.16,system,-23.2,SAVE,2022-08-31 00:00:00+02:00,1,9,14,7,57,2022-08-01,8
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,10400.0,train,3334.72,atm,-100.0,CHEQ,2022-07-30 00:00:00+02:00,0,12,11,1,29,2022-07-01,7


In [44]:
data.shape

(5144, 15)

In [45]:
train.shape, test.shape

((3602, 14), (1542, 14))

In [41]:
idkjkd

NameError: name 'idkjkd' is not defined

In [ ]:
ss.shape

In [ ]:
col2drop = ["CUSTOMER_IDENTIFIER", "DECLARED_NET_INCOME"]

In [ ]:
X = train.drop(col2drop, axis=1)
y = train["DECLARED_NET_INCOME"]

In [ ]:
test = test.drop(col2drop, axis=1)
test.head()

In [ ]:
X.shape

In [ ]:
test.head()

In [ ]:
X.dtypes

In [ ]:
test.dtypes

In [ ]:
y.isna().sum()

In [ ]:
X["PRODUCT_CODE_COUNT"] = X["PRODUCT_CODE"].map(
    X["PRODUCT_CODE"].value_counts().to_dict()
)
X["CHANNEL_COUNT"] = X["CHANNEL"].map(X["CHANNEL"].value_counts().to_dict())

test["PRODUCT_CODE_COUNT"] = test["PRODUCT_CODE"].map(
    test["PRODUCT_CODE"].value_counts().to_dict()
)
test["CHANNEL_COUNT"] = test["CHANNEL"].map(test["CHANNEL"].value_counts().to_dict())

In [ ]:
from sklearn.preprocessing import LabelEncoder

X["PRODUCT_CODE"] = LabelEncoder().fit_transform(X["PRODUCT_CODE"])
X["CHANNEL"] = LabelEncoder().fit_transform(X["CHANNEL"])

In [ ]:
test["PRODUCT_CODE"] = LabelEncoder().fit_transform(test["PRODUCT_CODE"])
test["CHANNEL"] = LabelEncoder().fit_transform(test["CHANNEL"])

In [ ]:
X.head()

In [ ]:
X.shape, test.shape

**Note** <br>
With Poisson regression and learning rate of 0.01 and max-depth of 8 score is 6455.224621

In [ ]:
params = {
    "objective": "tweedie",
    "metric": "rmse",
    "learning_rate": 0.01,
    "sub_feature": 0.8,
    "sub_row": 0.75,
    "bagging_freq": 1,
    "lambda_l2": 0.6,
    "verbosity": 0,
    "num_iterations": 7000,
    "num_leaves": 128,
    "min_data_in_leaf": 50,
    "max_depth": 9,
}

In [ ]:
%%time
errcb = []
y_pred_totcb = []
fold = KFold(n_splits=10, shuffle=True, random_state=42)
i = 1
for train_index, test_index in fold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m = LGBMRegressor(**params)
    # m = CatBoostRegressor()
    # m = XGBRegressor()
    m.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100,
        early_stopping_rounds=200,
    )
    preds = m.predict(X_test)
    print("err: ", metrics.mean_squared_error(y_test, preds, squared=False))
    errcb.append(metrics.mean_squared_error(y_test, preds, squared=False))
    p = m.predict(test)
    y_pred_totcb.append(p)

In [ ]:
print("The error of the model is: ", np.mean(errcb))

In [ ]:
fea_imp = pd.DataFrame({"imp": m.feature_importances_, "col": X.columns})
fea_imp = fea_imp.sort_values(["imp", "col"], ascending=False).iloc[-30:]
_ = fea_imp.plot(kind="barh", x="col", y="imp", figsize=(20, 10))
plt.savefig("catboost_feature_importance.png")

In [ ]:
np.mean(y_pred_totcb, 0)

In [ ]:
len(np.mean(y_pred_totcb, 0))

In [ ]:
test["DECLARED_NET_INCOME"] = np.mean(y_pred_totcb, 0)

In [ ]:
ss.head()

In [ ]:
ss["DECLARED_NET_INCOME"] = np.mean(y_pred_totcb, 0)
ss.head()

### transforming the prediction


In [ ]:
# ss["DECLARED_NET_INCOME"] = np.log1p(abs(ss["DECLARED_NET_INCOME"]))

In [ ]:
ss.head()

In [ ]:
ss.to_csv("base_lgb.csv", index=False)